In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load The Dataset

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_parquet("hf://datasets/SKNahin/bengali-transliteration-data/data/train-00000-of-00001.parquet")

# Split the dataset into 80% training and 20% validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Inspect the splits
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")


Training samples: 4004
Validation samples: 1002


# Data Preprocessing

In [7]:
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50")

# Filter overly short and long sentences
def filter_sentences(df):
    return df[(df['rm'].str.len() > 5) & (df['bn'].str.len() > 5) & 
              (df['rm'].str.len() <= 256) & (df['bn'].str.len() <= 256)]

train_df = filter_sentences(train_df)
val_df = filter_sentences(val_df)

# Tokenize the dataset
def tokenize_data(df):
    inputs = tokenizer(list(df['rm']), max_length=64, padding="max_length", truncation=True, return_tensors="pt")
    targets = tokenizer(list(df['bn']), max_length=64, padding="max_length", truncation=True, return_tensors="pt")
    return {"input_ids": inputs['input_ids'], "attention_mask": inputs['attention_mask'], "labels": targets['input_ids']}

train_data = tokenize_data(train_df)
val_data = tokenize_data(val_df)


tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Model Mbart

In [8]:
from transformers import MBartForConditionalGeneration

# Load mBART model
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [10]:
pip install wandb


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [12]:
import wandb
wandb.login(key="86d27f6971f8d9777dab3fbc093dc67fd8e3a515")



wandb: Currently logged in as: sakibmohammedsobaha (sakibmohammedsobaha-bangladesh-university-of-engineering). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
pip install evaluate


In [ ]:
pip install sacrebleu

# Train Using mbart

In [ ]:
# from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
# import torch
# import evaluate

# # Convert data to PyTorch Dataset
# class BanglishToBanglaDataset(torch.utils.data.Dataset):
#     def __init__(self, data):
#         self.input_ids = data["input_ids"]
#         self.attention_mask = data["attention_mask"]
#         self.labels = data["labels"]

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         return {
#             "input_ids": self.input_ids[idx],
#             "attention_mask": self.attention_mask[idx],
#             "labels": self.labels[idx],
#         }


# # Load BLEU metric
# bleu = evaluate.load("sacrebleu")

# # Evaluate on validation set
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Decode predictions and labels
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
#     # Format labels to match BLEU input requirements
#     formatted_labels = [[label] for label in decoded_labels]
#     # Compute BLEU score
#     bleu_score = bleu.compute(predictions=decoded_preds, references=formatted_labels)
#     return {"bleu": bleu_score["score"]}

# train_dataset = BanglishToBanglaDataset(train_data)
# val_dataset = BanglishToBanglaDataset(val_data)

# # Define training arguments
# training_args = Seq2SeqTrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     save_strategy="epoch",
#     logging_dir="./logs",
#     predict_with_generate=True,
#     load_best_model_at_end=True,
# )

# # Initialize Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,  # Include the metric computation
# )

# # Train the model
# trainer.train()


# Evaluate

In [ ]:




# trainer.evaluate()

# model.save_pretrained("./fine_tuned_mbart")
# tokenizer.save_pretrained("./fine_tuned_mbart")


In [ ]:
pip install datasets


In [13]:
pip show transformers


Name: transformers
Version: 4.44.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments
Note: you may need to restart the kernel to use updated packages.


# Train Using ByT5

In [ ]:
from transformers import ByT5Tokenizer, ByT5ForConditionalGeneration, Trainer, TrainingArguments
import evaluate
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Split into train and validation sets (80/20 split)
train_texts_t5, val_texts_t5, train_labels_t5, val_labels_t5 = train_test_split(
    df['rm'], df['bn'], test_size=0.2, random_state=42
)

# Create Hugging Face Dataset objects for ByT5
train_dataset_t5 = Dataset.from_dict({"input_text": train_texts_t5, "target_text": train_labels_t5})
val_dataset_t5 = Dataset.from_dict({"input_text": val_texts_t5, "target_text": val_labels_t5})

# Load pre-trained tokenizer and model for ByT5
model_name_t5 = "google/byt5-small"  # Use "google/byt5-small" for ByT5
tokenizer_t5 = ByT5Tokenizer.from_pretrained(model_name_t5)
model_t5 = ByT5ForConditionalGeneration.from_pretrained(model_name_t5)

# Preprocessing function for tokenization
def preprocess_function_t5(examples):
    inputs = [f"transliterate: {text}" for text in examples["input_text"]]
    targets = examples["target_text"]
    model_inputs = tokenizer_t5(inputs, max_length=128, truncation=True)
    labels = tokenizer_t5(targets, max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the datasets for ByT5
train_dataset_t5 = train_dataset_t5.map(preprocess_function_t5, batched=True)
val_dataset_t5 = val_dataset_t5.map(preprocess_function_t5, batched=True)

# Define training arguments for ByT5
training_args_t5 = TrainingArguments(
    output_dir="./results_byt5",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs_byt5",
    logging_steps=100,
    report_to="none",  # Disable W&B logging if needed
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Load BLEU metric
bleu = evaluate.load("sacrebleu")

# Compute BLEU score during evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode the predictions and labels to text
    decoded_preds = tokenizer_t5.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer_t5.batch_decode(labels, skip_special_tokens=True)
    
    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    
    # Calculate loss (using the `eval_loss` returned from Trainer)
    eval_loss = eval_pred.metrics["eval_loss"]
    
    return {
        "eval_loss": eval_loss,
        "bleu": bleu_score["score"],
    }

# Initialize Trainer for ByT5 with compute_metrics
trainer_t5 = Trainer(
    model=model_t5,
    args=training_args_t5,
    train_dataset=train_dataset_t5,
    eval_dataset=val_dataset_t5,
    tokenizer=tokenizer_t5,
    compute_metrics=compute_metrics  # Add this line to compute BLEU score and loss
)

# Train the model for ByT5
trainer_t5.train()

# Save the model for ByT5
model_t5.save_pretrained("./banglish_to_bangla_model_byt5")
tokenizer_t5.save_pretrained("./banglish_to_bangla_model_byt5")

# You can also manually evaluate the model after training
results_t5 = trainer_t5.evaluate()

# Print evaluation results
print(f"ByT5 Evaluation results: {results_t5}")


# Train using T5

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
# import evaluate

# # Split into train and validation sets (80/20 split)
# train_texts_t5, val_texts_t5, train_labels_t5, val_labels_t5 = train_test_split(
#     df['rm'], df['bn'], test_size=0.2, random_state=42
# )

# # Create Hugging Face Dataset objects for T5/mT5
# train_dataset_t5 = Dataset.from_dict({"input_text": train_texts_t5, "target_text": train_labels_t5})
# val_dataset_t5 = Dataset.from_dict({"input_text": val_texts_t5, "target_text": val_labels_t5})

# # Load pre-trained tokenizer and model for T5/mT5
# model_name_t5 = "google/mt5-small"  # Use "t5-small" for T5 or "google/mt5-small" for mT5
# tokenizer_t5 = T5Tokenizer.from_pretrained(model_name_t5)
# model_t5 = T5ForConditionalGeneration.from_pretrained(model_name_t5)

# # Preprocessing function for tokenization
# def preprocess_function_t5(examples):
#     inputs = [f"transliterate: {text}" for text in examples["input_text"]]
#     targets = examples["target_text"]
#     model_inputs = tokenizer_t5(inputs, max_length=128, truncation=True)
#     labels = tokenizer_t5(targets, max_length=128, truncation=True)

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # Tokenize the datasets for T5/mT5
# train_dataset_t5 = train_dataset_t5.map(preprocess_function_t5, batched=True)
# val_dataset_t5 = val_dataset_t5.map(preprocess_function_t5, batched=True)

# # Define training arguments for T5/mT5
# training_args_t5 = TrainingArguments(
#     output_dir="./results_t5",
#     evaluation_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_strategy="epoch",
#     logging_dir="./logs_t5",
#     logging_steps=100,
#     report_to="none",  # Disable W&B logging if needed
#     load_best_model_at_end=True,
#     metric_for_best_model="eval_loss",
# )

# # Load BLEU metric
# bleu = evaluate.load("sacrebleu")

# # Compute BLEU score during evaluation
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Decode the predictions and labels to text
#     decoded_preds = tokenizer_t5.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer_t5.batch_decode(labels, skip_special_tokens=True)
    
#     # Calculate BLEU score
#     bleu_score = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    
#     # Calculate loss (using the `eval_loss` returned from Trainer)
#     eval_loss = eval_pred.metrics["eval_loss"]
    
#     return {
#         "eval_loss": eval_loss,
#         "bleu": bleu_score["score"],
#     }

# # Initialize Trainer for T5/mT5 with compute_metrics
# trainer_t5 = Trainer(
#     model=model_t5,
#     args=training_args_t5,
#     train_dataset=train_dataset_t5,
#     eval_dataset=val_dataset_t5,
#     tokenizer=tokenizer_t5,
#     compute_metrics=compute_metrics  # Add this line to compute BLEU score and loss
# )

# # Train the model for T5/mT5
# trainer_t5.train()

# # Save the model for T5/mT5
# model_t5.save_pretrained("./banglish_to_bangla_model_t5")
# tokenizer_t5.save_pretrained("./banglish_to_bangla_model_t5")

# # You can also manually evaluate the model after training
# results_t5 = trainer_t5.evaluate()

# # Print evaluation results
# print(f"T5/mT5 Evaluation results: {results_t5}")


# Evaluate Metrics

In [ ]:


# # Load BLEU metric
# bleu = evaluate.load("sacrebleu")

# # Compute BLEU score during evaluation
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     # Decode the predictions and labels to text
#     decoded_preds = tokenizer_t5.batch_decode(predictions, skip_special_tokens=True)
#     decoded_labels = tokenizer_t5.batch_decode(labels, skip_special_tokens=True)
    
#     # Calculate BLEU score
#     bleu_score = bleu_metric.compute(predictions=decoded_preds, references=[[label] for label in decoded_labels])
    
#     # Calculate loss (using the `eval_loss` returned from Trainer)
#     eval_loss = eval_pred.metrics["eval_loss"]
    
#     return {
#         "eval_loss": eval_loss,
#         "bleu": bleu_score["score"],
#     }

# # Initialize Trainer for T5/mT5 with compute_metrics
# trainer_t5 = Trainer(
#     model=model_t5,
#     args=training_args_t5,
#     train_dataset=train_dataset_t5,
#     eval_dataset=val_dataset_t5,
#     tokenizer=tokenizer_t5,
#     compute_metrics=compute_metrics  # Add this line to compute BLEU score and loss
# )

# # Train the model for T5/mT5
# trainer_t5.train()

# # Save the model for T5/mT5
# model_t5.save_pretrained("./banglish_to_bangla_model_t5")
# tokenizer_t5.save_pretrained("./banglish_to_bangla_model_t5")

# # You can also manually evaluate the model after training
# results_t5 = trainer_t5.evaluate()

# # Print evaluation results
# print(f"T5/mT5 Evaluation results: {results_t5}")
